In [6]:
import nltk
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm
from wordcloud import WordCloud
import twint
import nest_asyncio

nest_asyncio.apply()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Raizel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')

# Inception

In [3]:
Inception = pd.read_csv('Inception.csv')
Inception.drop(columns = ['Unnamed: 0'], axis =1, inplace=True)
Inception.head()

,date,tweet,language,hashtags,username,link,retweet,nlikes,nreplies,nretweets,retweet_date,vader_score,positive sentiment score,negative sentiment score,neutral sentiment score,compound sentiment score
0,2011-07-11 19:59:41,Who understands that Inception movie?,en,[],debbie_nthejets,https://twitter.com/debbie_nthejets/status/905...,False,0,0,0,NaN,0.0000,0.000,0.0,1.000,0.0000
1,2011-07-11 19:59:29,Inception in my pants #improvefilmtitlesbyaddi...,en,['improvefilmtitlesbyaddinginmypants'],MercDeejay,https://twitter.com/MercDeejay/status/90570938...,False,0,0,0,NaN,0.0000,0.000,0.0,1.000,0.0000
2,2011-07-11 19:59:09,RT @ imNot_daFather #GreatHighMovie Inception,en,['greathighmovie'],Lejailatroi,https://twitter.com/Lejailatroi/status/9057085...,False,0,0,0,NaN,0.0000,0.000,0.0,1.000,0.0000
3,2011-07-11 19:58:23,never saw inception,en,[],6tacey,https://twitter.com/6tacey/status/905706630473...,False,0,0,1,NaN,0.0000,0.000,0.0,1.000,0.0000
4,2011-07-11 19:57:45,@jck49erfan @Cornholio3000 was inception the o...,en,[],JHScramble,https://twitter.com/JHScramble/status/90570505...,False,0,2,0,NaN,0.4215,0.123,0.0,0.877,0.4215


In [4]:
#tokenizing each tweet
tokenized_tweets = []
for tweet in Inception['tweet'].values:
    tokenized_tweets.append(tokenizer.tokenize(tweet.lower())) #making all tweets lowercase so no word is duplicated later on
    
print(tokenized_tweets)

[['who', 'understands', 'that', 'inception', 'movie'], ['inception', 'in', 'my', 'pants', 'improvefilmtitlesbyaddinginmypants'], ['rt', 'imnot', 'dafather', 'greathighmovie', 'inception'], ['never', 'saw', 'inception'], ['jck49erfan', 'cornholio3000', 'was', 'inception', 'the', 'one', 'where', 'leo', 'is', 'a', 'patient', 'and', 'the', 'ending', 'was', 'spoiled', 'on', 'a', 'popular', 'sport', 's', 'blog'], ['creamadoir', 'inception', 'in', 'my', 'pants', 'lol'], ['esmeraldabaza', 'inception', 'jkjk', 'lol'], ['sleeping', 'in', 'the', 'backseat', 'of', 'a', 'van', 'while', 'it', 's', 'in', 'motion', 'inception'], ['watching', 'adjustment', 'bureau', 'inception', 'with', 'trilbys'], ['inception', 'inception', 'http', 'bit', 'ly', 'nalu29'], ['warriorgrrrl', 'since', 'i', 've', 'watched', 'inception', 'it', 'happens', 'to', 'me', 'all', 'the', 'time', 'brilliant', 'movie', 'but', 'a', 'total', 'mind', 'fuck'], ['improvefilmtitlesbyaddinginmypants', 'inception', 'i', 'my', 'pants'], ['wat

In [7]:
#adding additional words I don't find to be that valuable
additional_stopwords = ['https', 'I', 'com', 'http', 'RT', 'co', 'the', 'amp', 'the', 'lt', 'A', 'www', 'rt', 'got', 'get', 
                        'ly', 'u']

stopwords_list = stopwords.words('english')

stopwords_list += list(string.punctuation)

stopwords_list += additional_stopwords

print(stopwords_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
#isolating every word within tweets for the Inception movie (not including stop words)
tweets = []
for tweet in tokenized_tweets:
    for word in tweet:
        if word not in stopwords_list:
            tweets.append(word)  

tweets

['understands',
 'inception',
 'movie',
 'inception',
 'pants',
 'improvefilmtitlesbyaddinginmypants',
 'imnot',
 'dafather',
 'greathighmovie',
 'inception',
 'never',
 'saw',
 'inception',
 'jck49erfan',
 'cornholio3000',
 'inception',
 'one',
 'leo',
 'patient',
 'ending',
 'spoiled',
 'popular',
 'sport',
 'blog',
 'creamadoir',
 'inception',
 'pants',
 'lol',
 'esmeraldabaza',
 'inception',
 'jkjk',
 'lol',
 'sleeping',
 'backseat',
 'van',
 'motion',
 'inception',
 'watching',
 'adjustment',
 'bureau',
 'inception',
 'trilbys',
 'inception',
 'inception',
 'bit',
 'nalu29',
 'warriorgrrrl',
 'since',
 'watched',
 'inception',
 'happens',
 'time',
 'brilliant',
 'movie',
 'total',
 'mind',
 'fuck',
 'improvefilmtitlesbyaddinginmypants',
 'inception',
 'pants',
 'watchinq',
 'inception',
 'lenardo',
 '3',
 'like',
 'inception',
 'inception',
 'hmmm',
 'amaaazing',
 'concept',
 'movie',
 'really',
 'liked',
 'read',
 'word',
 'magazines',
 'deejaylunchbox',
 'dream',
 'djexodus',
 '

In [9]:
ps = PorterStemmer()

stemmed_tweets=[]
for word in tweets:
    stemmed_tweets.append(ps.stem(word))

print(stemmed_tweets)

['understand', 'incept', 'movi', 'incept', 'pant', 'improvefilmtitlesbyaddinginmyp', 'imnot', 'dafath', 'greathighmovi', 'incept', 'never', 'saw', 'incept', 'jck49erfan', 'cornholio3000', 'incept', 'one', 'leo', 'patient', 'end', 'spoil', 'popular', 'sport', 'blog', 'creamadoir', 'incept', 'pant', 'lol', 'esmeraldabaza', 'incept', 'jkjk', 'lol', 'sleep', 'backseat', 'van', 'motion', 'incept', 'watch', 'adjust', 'bureau', 'incept', 'trilbi', 'incept', 'incept', 'bit', 'nalu29', 'warriorgrrrl', 'sinc', 'watch', 'incept', 'happen', 'time', 'brilliant', 'movi', 'total', 'mind', 'fuck', 'improvefilmtitlesbyaddinginmyp', 'incept', 'pant', 'watchinq', 'incept', 'lenardo', '3', 'like', 'incept', 'incept', 'hmmm', 'amaaaz', 'concept', 'movi', 'realli', 'like', 'read', 'word', 'magazin', 'deejaylunchbox', 'dream', 'djexodu', 'incept', 'dope', 'movi', 'incept', 'pant', 'improvefilmtitlesbyaddinginmyp', 'nw', 'incept', '1', '2', 'way', 'social', 'network', 'awesom', 'far', 'much', 'better', 'incep

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(stopwords_list, lowercase=True)
text_tf= tf.fit_transform(Inception['tweet'])
text_tf.data

array([0.32302826, 0.08677956, 0.33925085, ..., 0.21080881, 0.1114055 ,
       0.06127721])

In [ ]:
# set X features and y target
X = df_1['stemmed_tweets']
y = df_1['vader_score']

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

In [ ]:
# instantiating tf_idf vectorizer
tfidf = TfidfVectorizer(stop_words= stopwords_list, lowercase=True, ngram_range=(1,1))

In [ ]:
#tokenize and feature engineer using tfidf 
train_features = tfidf.fit_transform(X_train)
test_features = tfidf.transform(X_test)

In [ ]:
# step 3 - fit the model to the training set, test the model on the testing set
classifier = RandomForestClassifier()
classifier.fit(train_features,y_train)

In [ ]:
# step 4 - output prediction on testing set, compare true labels to predicted labels, get accuracy
pred = classifier.predict(test_features)

In [ ]:
accuracy_score(pred, y_test)

In [ ]:
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df['review'])